In [6]:
from typing import List, Dict, Literal, Union
import datetime, time 
import numpy as np 
import pandas as pd
import RPi.GPIO as GPIO
from multiprocessing import Pipe, Process
from multiprocessing.connection import Connection

from data_collection import Logger

In [7]:

def setup_pwm(pin, freq=300) -> GPIO.PWM:
    GPIO.setup(pin, GPIO.OUT)
    return GPIO.PWM(pin, freq) 

class TwoWheels:
    def __init__(self, ch_left, ch_right):
        ch_left.start(0)
        ch_right.start(0)
        self.ch_left = ch_left
        self.ch_right = ch_right
        
    def step(self, left: Union[None, int], right: Union[None, int]) -> None:
        
        if (left is None) and (right is None):
            return 
        
        self.ch_left.ChangeDutyCycle(left)
        self.ch_right.ChangeDutyCycle(right)

    def sample(input_con: Connection) -> (Union[None, int], Union[None, int]):
        """
        read only the latest input

        the component stale if the inputs are comming faster than it can read
        """
        left, right = None, None
        while input_con.poll():
            left, right = input_con.recv()
        return left, right
    
    
    def main(left_pin:int, right_pin:int, input_con: Connection, **logger_kwargs):
        
        GPIO.setmode(GPIO.BOARD)
        ch_left = setup_pwm(left_pin,freq=300)
        ch_right = setup_pwm(right_pin,freq=300)

        logger = Logger(**logger_kwargs)
        
        component = TwoWheels(ch_left, ch_right)
            
        while True:
            left, right = TwoWheels.sample(input_con)
            component.step(left, right)

            
            logger.increment_idx() # time is recorded here
            logger.log('left', left)
            logger.log('right', right)
            
            time.sleep(0.1)

    def start(left_pin, right_pin, **logger_kwargs):
        """
        using GPIO.BOARD pin
        """
        
        receiver, sender = Pipe(False)
        process = Process(target=TwoWheels.main, args=(left_pin, right_pin, receiver), kwargs=logger_kwargs)
        process.start()
        return process, sender

    

In [8]:
process, sender = TwoWheels.start(32, 33, name='two_wheels')

/tmp/ipykernel_3309/1605402927.py:2: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT)


In [5]:
sender.send((20,20))

In [6]:
sender.send((0,0))

In [9]:
Logger(overwrite_ok=True).load()

[]